In [1]:
import re
from pathlib import Path

def split_on_numeric(s):
    return list(filter(None, re.split(r'(\d+)', s)))


def post_process_name(name):
    return name.replace('/', '_').replace('\\', '_').strip()


def get_name_from_entry(entry):
    name = entry.split('\n')[0].split('{')[-1].split(',')[0]
    name = post_process_name(name)
    return name


def load_bib_entries_from_file(file, unallowed=['abstract', 'keywords', 'numpages', 'series', 'location']):
    bib_contents = None
    with open(file, 'r') as f:
        bib_contents = f.read()

    # Each item of this list will be a bibtex entry string
    bib_entries = [b.strip() for b in bib_contents.split('\n\n')]

    processed = []
    for entry in bib_entries:
        lines = [l for l in entry.split('\n') if sum([l.strip().lower().startswith(u) for u in unallowed]) == 0]
        processed.append('\n'.join(lines))
    bib_entries = processed
    bib_entries = {get_name_from_entry(b): b for b in bib_entries}
    return bib_entries


def write_bib_files(bib_entries, folder):
    folder = Path(folder)
    if not folder.exists():
        folder.mkdir(parents=True)
    
    for entry in bib_entries.keys():
        filepath = folder / Path('{}.tex'.format(entry))
        with open(filepath, 'w') as f:
            f.write(bib_entries[entry])


def create_tooltip_for_bib_entry(key, bib_entries):
    assert(key in bib_entries)

    bibtex_string = bib_entries[key]
    bibtex_string = bibtex_string.strip().replace('\n', '<br>')

    bibtex_filename = 'bibtex/{}.tex'.format(key)

    author_name, paper_year = split_on_numeric(key)[1:3]
    author_name = author_name.capitalize()
    paper_string = '{} ({})'.format(author_name, paper_year)

    return '''<p class="tooltip"><a href="{}" class="grey-text text-darken-3">{}</a><span class="tooltiptext">{}</span></p>'''.format(bibtex_filename, paper_string, bibtex_string)

In [2]:
bib_entries = load_bib_entries_from_file('references_for_tables.bib')
write_bib_files(bib_entries, 'bibtex')

In [3]:
def comma_sep_str_to_html(string, bib_entries):
    keys = [post_process_name(n.strip()) for n in string.split(',')]
    result = ""
    for index in range(len(keys)):
        #print(keys[index] )
        assert(keys[index] in bib_entries.keys())
        result += create_tooltip_for_bib_entry(keys[index], bib_entries)
        if index != len(keys) - 1:
            result += ", "
    return result

In [ ]:
s = '019zhang2023boss'

In [ ]:
print(comma_sep_str_to_html(s, bib_entries))